# 🚀 NetBird Python Client Demo

This notebook demonstrates how to use the NetBird Python client to manage your NetBird network.

## 📋 What you'll learn:
- ✅ Connect to NetBird API
- ✅ Manage users, peers, and groups
- ✅ Configure policies and routes
- ✅ Monitor network activity
- ✅ Handle errors gracefully

## 🔧 Setup

First, let's import the NetBird client and set up authentication.

In [30]:
# Import the NetBird client
from netbird import APIClient
from netbird.exceptions import NetBirdAPIError, NetBirdNotFoundError
import os
from pprint import pprint
import json

print("✅ NetBird Python client imported successfully!")

✅ NetBird Python client imported successfully!


In [31]:
# Set up your NetBird credentials
# Option 1: Set environment variables (recommended)
NETBIRD_HOST = os.getenv("NETBIRD_HOST", "api.netbird.io")
NETBIRD_TOKEN = os.getenv("NETBIRD_API_TOKEN")

# Option 2: Set directly (for demo only - don't commit tokens!)
# NETBIRD_HOST = "your-netbird-server.com"
# NETBIRD_TOKEN = "your-api-token-here"

if not NETBIRD_TOKEN:
    print("⚠️  No API token found. Set NETBIRD_API_TOKEN environment variable.")
    print("   Example: export NETBIRD_API_TOKEN='your-token-here'")
else:
    print(f"✅ Using NetBird server: {NETBIRD_HOST}")
    print(f"✅ Token configured: {NETBIRD_TOKEN[:10]}...")

✅ Using NetBird server: https://netbird.oci.cloudpoet.in:33073
✅ Token configured: nbp_x3XhFU...


In [32]:
import requests
import json

url = f"{NETBIRD_HOST}/api/users/current"

headers = {     
  'Accept': 'application/json',
  'Authorization': f'Token {NETBIRD_TOKEN}'
}

response = requests.request("GET", url, headers=headers)

(response.json())


{'auto_groups': [],
 'email': 'bhushan.rane@cloudpoet.in',
 'id': '103489590992358909976',
 'is_blocked': False,
 'is_current': True,
 'is_service_user': False,
 'issued': 'api',
 'last_login': '2025-07-21T16:56:05.42356783Z',
 'name': 'Bhushan Rane',
 'permissions': {'is_restricted': False,
  'modules': {'accounts': {'create': True,
    'delete': True,
    'read': True,
    'update': True},
   'dns': {'create': True, 'delete': True, 'read': True, 'update': True},
   'events': {'create': True, 'delete': True, 'read': True, 'update': True},
   'groups': {'create': True, 'delete': True, 'read': True, 'update': True},
   'nameservers': {'create': True,
    'delete': True,
    'read': True,
    'update': True},
   'networks': {'create': True, 'delete': True, 'read': True, 'update': True},
   'pats': {'create': True, 'delete': True, 'read': True, 'update': True},
   'peers': {'create': True, 'delete': True, 'read': True, 'update': True},
   'policies': {'create': True, 'delete': True, 'read

## 🌐 Connect to NetBird

Create a client connection to your NetBird server.

In [33]:
# Create NetBird client
client = APIClient(
    host=NETBIRD_HOST,
    api_token=NETBIRD_TOKEN,
    timeout=30.0
)

print(f"🔗 Connected to: {client.base_url}")
print(f"🔐 Authentication: Token configured")
print(f"⏱️  Timeout: {client.timeout} seconds")

🔗 Connected to: https://netbird.oci.cloudpoet.in:33073/api
🔐 Authentication: Token configured
⏱️  Timeout: 30.0 seconds


## 👤 User Management

Let's start by exploring users in your NetBird network.

In [34]:
# Get current user information
try:
    current_user = client.users.get_current()
    print("👤 Current User:")
    print(f"   Name: {current_user['name']}")
    print(f"   Email: {current_user['email']}")
    print(f"   Role: {current_user['role']}")
    print(f"   Status: {current_user['status']}")
    print(f"   Last Login: {current_user['last_login']}")
except NetBirdAPIError as e:
    print(f"❌ Error getting current user: {e}")

👤 Current User:
   Name: Bhushan Rane
   Email: bhushan.rane@cloudpoet.in
   Role: owner
   Status: active
   Last Login: 2025-07-21T16:56:05.42356783Z


In [35]:
# List all users
try:
    users = client.users.list()
    print(f"👥 Found {len(users)} users:")
    print()
    
    for user in users:
        print(f"   📧 {user['email'] or 'No email'}")
        print(f"      Role: {user['role']} | Status: {user['status']}")
        print(f"      Service User: {user['is_service_user']} | Blocked: {user['is_blocked']}")
        print()
        
except NetBirdAPIError as e:
    print(f"❌ Error listing users: {e}")

👥 Found 4 users:

   📧 bhushan.rane@cloudpoet.in
      Role: owner | Status: active
      Service User: False | Blocked: False

   📧 No email
      Role: admin | Status: active
      Service User: True | Blocked: False

   📧 shreya.fegade@cloudpoet.in
      Role: user | Status: active
      Service User: False | Blocked: False

   📧 mahendra.bendale@cloudpoet.in
      Role: admin | Status: active
      Service User: False | Blocked: False



## 🖥️ Peer Management

Explore the devices (peers) connected to your NetBird network.

In [36]:
# List all peers
try:
    peers = client.peers.list()
    print(f"🖥️  Found {len(peers)} peers:")
    print()
    
    for peer in peers:
        status = "🟢 Online" if peer['connected'] else "🔴 Offline"
        print(f"   {status} {peer['name']}")
        print(f"      IP: {peer['ip']} | OS: {peer['os']}")
        print(f"      Version: {peer['version']} | Last Seen: {peer['last_seen']}")
        
        # Show geographic info if available
        if peer.get('city_name') and peer.get('country_code'):
            print(f"      Location: {peer['city_name']}, {peer['country_code']}")
        
        # Show groups
        if peer.get('groups'):
            group_names = [g.get('name', 'Unknown') for g in peer['groups']]
            print(f"      Groups: {', '.join(group_names)}")
        
        print()
        
except NetBirdAPIError as e:
    print(f"❌ Error listing peers: {e}")

🖥️  Found 4 peers:

   🟢 Online rb-oci-k8s
      IP: 100.107.174.91 | OS: Alpine Linux 3.21.3
      Version: 0.50.1 | Last Seen: 2025-07-15T07:44:28.888472203Z
      Location: Mumbai, IN
      Groups: All, rb-oci

   🔴 Offline BHUSHAN-RANE.local
      IP: 100.107.206.151 | OS: Darwin 15.5
      Version: 0.50.0 | Last Seen: 2025-07-21T17:09:45.41439433Z
      Location: Pune, IN
      Groups: All

   🔴 Offline DESKTOP-I6AVTGU
      IP: 100.107.114.89 | OS: Windows 10
      Version: 0.50.2 | Last Seen: 2025-07-11T16:26:57.190228067Z
      Location: Pune, IN
      Groups: All

   🟢 Online homelab-docker
      IP: 100.107.146.58 | OS: Alpine Linux 3.21.3
      Version: 0.50.1 | Last Seen: 2025-07-26T23:25:28.771849315Z
      Location: Pune, IN
      Groups: All, homelab



## 👥 Group Management

Manage groups to organize your network peers.

In [37]:
# List all groups
try:
    groups = client.groups.list()
    print(f"👥 Found {len(groups)} groups:")
    print()
    
    for group in groups:
        print(f"   📁 {group['name']}")
        print(f"      ID: {group['id']}")
        print(f"      Peers: {group['peers_count']}")
        print(f"      Resources: {group.get('resources_count', 0)}")
        
        # Show peer details if available
        if group.get('peers') and len(group['peers']) <= 5:  # Don't show too many
            peer_names = [p.get('name', 'Unknown') for p in group['peers']]
            print(f"      Members: {', '.join(peer_names)}")
        
        print()
        
except NetBirdAPIError as e:
    print(f"❌ Error listing groups: {e}")

👥 Found 4 groups:

   📁 All
      ID: d1ip995aofms73a23dp0
      Peers: 4
      Resources: 1
      Members: rb-oci-k8s, BHUSHAN-RANE.local, DESKTOP-I6AVTGU, homelab-docker

   📁 ac-oci
      ID: d1mclvtaofms738ms6ng
      Peers: 0
      Resources: 1

   📁 homelab
      ID: d1mcm7taofms738ms6og
      Peers: 1
      Resources: 1
      Members: homelab-docker

   📁 rb-oci
      ID: d1mcn8daofms738ms6pg
      Peers: 1
      Resources: 2
      Members: rb-oci-k8s



## 🔐 Policy Management

View and manage network access policies.

In [38]:
# List all policies
try:
    policies = client.policies.list()
    print(f"🔐 Found {len(policies)} policies:")
    print()
    
    for policy in policies:
        status = "✅ Enabled" if policy['enabled'] else "❌ Disabled"
        print(f"   {status} {policy['name']}")
        print(f"      Description: {policy.get('description') or 'No description'}")
        print(f"      Rules: {len(policy.get('rules', []))}")
        
        # Show rule details
        if policy.get('rules'):
            for rule in policy['rules'][:2]:  # Show first 2 rules
                action = "🟢 Allow" if rule.get('action') == 'accept' else "🔴 Deny"
                print(f"        {action} {rule.get('name', 'Unnamed rule')}")
                print(f"          Protocol: {rule.get('protocol', 'any')}")
                
                # Show sources and destinations
                sources = rule.get('sources', [])
                destinations = rule.get('destinations', [])
                if sources:
                    src_names = [s.get('name', 'Unknown') for s in sources[:2]]
                    print(f"          From: {', '.join(src_names)}")
                if destinations:
                    dst_names = [d.get('name', 'Unknown') for d in destinations[:2]]
                    print(f"          To: {', '.join(dst_names)}")
        
        print()
        
except NetBirdAPIError as e:
    print(f"❌ Error listing policies: {e}")

🔐 Found 4 policies:

   ✅ Enabled Default
      Description: This is a default rule that allows connections between all the resources
      Rules: 1
        🟢 Allow Default
          Protocol: all
          From: All
          To: All

   ✅ Enabled admins
      Description: No description
      Rules: 1
        🟢 Allow admins
          Protocol: all
          From: All
          To: ac-oci, homelab

   ✅ Enabled test-policy-a40116dd
      Description: Integration test policy
      Rules: 1
        🟢 Allow Allow all
          Protocol: all

   ✅ Enabled test-policy-94e9b44d
      Description: Integration test policy
      Rules: 1
        🟢 Allow Allow all
          Protocol: all



## 🌐 Network Information

View network configurations and routes.

In [39]:
# List networks
try:
    networks = client.networks.list()
    print(f"🌐 Found {len(networks)} networks:")
    print()
    
    for network in networks:
        print(f"   📡 {network['name']}")
        print(f"      Description: {network.get('description') or 'No description'}")
        print(f"      Routers: {len(network.get('routers', []))}")
        print(f"      Resources: {len(network.get('resources', []))}")
        print(f"      Policies: {len(network.get('policies', []))}")
        print(f"      Routing Peers: {network.get('routing_peers_count', 0)}")
        print()
        
except NetBirdAPIError as e:
    print(f"❌ Error listing networks: {e}")

🌐 Found 3 networks:

   📡 ac-oracle-cloud
      Description: No description
      Routers: 1
      Resources: 1
      Policies: 1
      Routing Peers: 0

   📡 homelab
      Description: No description
      Routers: 1
      Resources: 1
      Policies: 1
      Routing Peers: 1

   📡 rb-oracle-cloud
      Description: No description
      Routers: 1
      Resources: 2
      Policies: 2
      Routing Peers: 1



In [19]:
# List routes
try:
    routes = client.routes.list()
    print(f"🛣️  Found {len(routes)} routes:")
    
    if routes:
        print()
        for route in routes:
            status = "✅ Enabled" if route['enabled'] else "❌ Disabled"
            print(f"   {status} {route.get('description') or 'Unnamed route'}")
            print(f"      Network: {route['network_id']}")
            print(f"      Metric: {route['metric']}")
            print()
    else:
        print("   No routes configured.")
        
except NetBirdAPIError as e:
    print(f"❌ Error listing routes: {e}")

🛣️  Found 0 routes:
   No routes configured.


## 🔑 Setup Keys

Manage setup keys for adding new devices.

In [20]:
# List setup keys
try:
    setup_keys = client.setup_keys.list()
    print(f"🔑 Found {len(setup_keys)} setup keys:")
    print()
    
    for key in setup_keys:
        status = "✅ Valid" if key['valid'] and not key['revoked'] else "❌ Invalid"
        print(f"   {status} {key['name']}")
        print(f"      Type: {key['type']}")
        print(f"      Used: {key['used_times']} times")
        print(f"      Limit: {key.get('usage_limit') or 'Unlimited'}")
        print(f"      Expires: {key.get('expires') or 'Never'}")
        print(f"      Last Used: {key.get('last_used') or 'Never'}")
        
        if key.get('auto_groups'):
            print(f"      Auto Groups: {len(key['auto_groups'])} groups")
        
        print()
        
except NetBirdAPIError as e:
    print(f"❌ Error listing setup keys: {e}")

🔑 Found 3 setup keys:

   ✅ Valid rb-oracle-cloud
      Type: reusable
      Used: 1 times
      Limit: Unlimited
      Expires: 0001-01-01T00:00:00Z
      Last Used: 2025-07-08T07:45:45.878095804Z
      Auto Groups: 1 groups

   ✅ Valid ac-oracle-cloud
      Type: reusable
      Used: 3 times
      Limit: Unlimited
      Expires: 0001-01-01T00:00:00Z
      Last Used: 2025-07-08T08:02:03.433632539Z
      Auto Groups: 1 groups

   ✅ Valid homelab
      Type: reusable
      Used: 2 times
      Limit: Unlimited
      Expires: 0001-01-01T00:00:00Z
      Last Used: 2025-07-24T11:26:22.631984411Z
      Auto Groups: 1 groups



## 🏢 Account Information

View account settings and configuration.

In [21]:
# List accounts
try:
    accounts = client.accounts.list()
    print(f"🏢 Found {len(accounts)} accounts:")
    print()
    
    for account in accounts:
        print(f"   🏷️  {account['domain']}")
        print(f"      Category: {account['domain_category']}")
        print(f"      Created: {account['created_at']}")
        print(f"      Created By: {account['created_by']}")
        
        # Show settings if available
        if account.get('settings'):
            settings = account['settings']
            print(f"      Settings:")
            
            # Show key settings
            if isinstance(settings, dict):
                if 'peer_login_expiration_enabled' in settings:
                    print(f"        Login Expiration: {settings['peer_login_expiration_enabled']}")
                if 'groups_propagation_enabled' in settings:
                    print(f"        Group Propagation: {settings['groups_propagation_enabled']}")
                if 'lazy_connection_enabled' in settings:
                    print(f"        Lazy Connection: {settings['lazy_connection_enabled']}")
        
        print()
        
except NetBirdAPIError as e:
    print(f"❌ Error listing accounts: {e}")

🏢 Found 1 accounts:

   🏷️  netbird.oci.cloudpoet.in
      Category: private
      Created: 2025-07-02T20:20:52.230069272Z
      Created By: 103489590992358909976
      Settings:
        Login Expiration: True
        Group Propagation: True
        Lazy Connection: False



## 📊 Activity & Events

Monitor network activity and audit events.

In [22]:
# Get recent audit events
try:
    events = client.events.get_audit_events()
    print(f"📊 Found {len(events)} recent events:")
    print()
    
    # Show last 5 events
    for event in events[:5]:
        print(f"   📅 {event['timestamp']}")
        print(f"      Activity: {event['activity']}")
        
        if event.get('initiator_name'):
            print(f"      Initiator: {event['initiator_name']}")
        if event.get('initiator_email'):
            print(f"      Email: {event['initiator_email']}")
        
        print()
        
except NetBirdAPIError as e:
    print(f"❌ Error getting events: {e}")

📊 Found 594 recent events:

   📅 2025-07-28T00:20:09.234574696Z
      Activity: Group deleted
      Initiator: Bhushan Rane
      Email: bhushan.rane@cloudpoet.in

   📅 2025-07-28T00:20:08.718745583Z
      Activity: Group created
      Initiator: Bhushan Rane
      Email: bhushan.rane@cloudpoet.in

   📅 2025-07-28T00:20:08.63418721Z
      Activity: Group deleted
      Initiator: Bhushan Rane
      Email: bhushan.rane@cloudpoet.in

   📅 2025-07-28T00:20:08.594269606Z
      Activity: Group created
      Initiator: Bhushan Rane
      Email: bhushan.rane@cloudpoet.in

   📅 2025-07-28T00:20:06.98574139Z
      Activity: Group deleted
      Initiator: Bhushan Rane
      Email: bhushan.rane@cloudpoet.in



## 🔧 DNS Management

Configure DNS settings for your network.

In [23]:
# Get DNS settings
try:
    dns_settings = client.dns.get_settings()
    print("🔧 DNS Settings:")
    print(f"   Disabled Management Groups: {len(dns_settings.get('disabled_management_groups', []))}")
    print()
    
except NetBirdAPIError as e:
    print(f"❌ Error getting DNS settings: {e}")
    print()

🔧 DNS Settings:
   Disabled Management Groups: 0



In [24]:
# List DNS nameserver groups
try:
    nameserver_groups = client.dns.list_nameserver_groups()
    print(f"🌐 Found {len(nameserver_groups)} DNS nameserver groups:")
    
    if nameserver_groups:
        print()
        for group in nameserver_groups:
            status = "✅ Enabled" if group['enabled'] else "❌ Disabled"
            print(f"   {status} {group['name']}")
            print(f"      Nameservers: {', '.join(group['nameservers'])}")
            if group.get('description'):
                print(f"      Description: {group['description']}")
            print()
    else:
        print("   No DNS nameserver groups configured.")
        
except NetBirdAPIError as e:
    print(f"❌ Error listing DNS nameserver groups: {e}")

🌐 Found 0 DNS nameserver groups:
   No DNS nameserver groups configured.


## 🎯 Create/Update Operations

Demonstrate how to create and update resources (use with caution!).

In [25]:
# Example: Create a new group (uncomment to run)
from netbird.models import GroupCreate

# UNCOMMENT THE LINES BELOW TO CREATE A TEST GROUP:
try:
    # Create a test group
    group_data = GroupCreate(
        name="Demo-Group-from-Jupyter",
        peers=[]  # Start with no peers
    )
    
    new_group = client.groups.create(group_data)
    print(f"✅ Created group: {new_group['name']} (ID: {new_group['id']})")
    
    # Clean up - delete the group immediately
    client.groups.delete(new_group['id'])
    print(f"🗑️  Deleted test group: {new_group['name']}")
    
except NetBirdAPIError as e:
    print(f"❌ Error with group operations: {e}")

print("💡 Uncomment the code above to test group creation/deletion")

✅ Created group: Demo-Group-from-Jupyter (ID: d23depdaofms73c1pfag)
🗑️  Deleted test group: Demo-Group-from-Jupyter
💡 Uncomment the code above to test group creation/deletion


## 🛡️ Error Handling Examples

Demonstrate proper error handling patterns.

In [26]:
# Example: Handling different types of errors
from netbird.exceptions import (
    NetBirdAPIError,
    NetBirdNotFoundError,
    NetBirdAuthenticationError,
    NetBirdValidationError
)

def safe_api_call(operation_name, api_function, *args, **kwargs):
    """Safely execute an API call with proper error handling."""
    try:
        result = api_function(*args, **kwargs)
        print(f"✅ {operation_name}: Success")
        return result
    
    except NetBirdAuthenticationError as e:
        print(f"🔐 {operation_name}: Authentication failed - check your API token")
        return None
    
    except NetBirdNotFoundError as e:
        print(f"🔍 {operation_name}: Resource not found")
        return None
    
    except NetBirdValidationError as e:
        print(f"📝 {operation_name}: Validation error - {e}")
        return None
    
    except NetBirdAPIError as e:
        print(f"⚠️  {operation_name}: API error (status {e.status_code}) - {e}")
        return None
    
    except Exception as e:
        print(f"💥 {operation_name}: Unexpected error - {e}")
        return None

# Test error handling
print("🧪 Testing error handling:")

# This should fail - invalid group ID
result = safe_api_call(
    "Get invalid group",
    client.groups.get,
    "invalid-group-id-12345"
)

# This should succeed - list users
result = safe_api_call(
    "List users",
    client.users.list
)

print("\n💡 Use this pattern in your production code!")

🧪 Testing error handling:
🔍 Get invalid group: Resource not found
✅ List users: Success

💡 Use this pattern in your production code!


## 📈 Network Statistics Summary

Get a complete overview of your NetBird network.

In [27]:
# Generate network summary
print("📈 NetBird Network Summary")
print("=" * 50)

summary = {}

# Collect statistics
try:
    users = client.users.list()
    summary['total_users'] = len(users)
    summary['admin_users'] = len([u for u in users if u['role'] == 'admin'])
    summary['active_users'] = len([u for u in users if u['status'] == 'active'])
except:
    summary['total_users'] = 'Error'

try:
    peers = client.peers.list()
    summary['total_peers'] = len(peers)
    summary['online_peers'] = len([p for p in peers if p['connected']])
    summary['offline_peers'] = len([p for p in peers if not p['connected']])
except:
    summary['total_peers'] = 'Error'

try:
    groups = client.groups.list()
    summary['total_groups'] = len(groups)
except:
    summary['total_groups'] = 'Error'

try:
    policies = client.policies.list()
    summary['total_policies'] = len(policies)
    summary['enabled_policies'] = len([p for p in policies if p['enabled']])
except:
    summary['total_policies'] = 'Error'

try:
    routes = client.routes.list()
    summary['total_routes'] = len(routes)
except:
    summary['total_routes'] = 'Error'

try:
    setup_keys = client.setup_keys.list()
    summary['total_setup_keys'] = len(setup_keys)
    summary['valid_setup_keys'] = len([k for k in setup_keys if k['valid'] and not k['revoked']])
except:
    summary['total_setup_keys'] = 'Error'

# Display summary
print(f"👥 Users:          {summary.get('total_users', 'N/A')} total, {summary.get('admin_users', 'N/A')} admins, {summary.get('active_users', 'N/A')} active")
print(f"🖥️  Peers:          {summary.get('total_peers', 'N/A')} total, {summary.get('online_peers', 'N/A')} online, {summary.get('offline_peers', 'N/A')} offline")
print(f"👥 Groups:         {summary.get('total_groups', 'N/A')}")
print(f"🔐 Policies:       {summary.get('total_policies', 'N/A')} total, {summary.get('enabled_policies', 'N/A')} enabled")
print(f"🛣️  Routes:         {summary.get('total_routes', 'N/A')}")
print(f"🔑 Setup Keys:     {summary.get('total_setup_keys', 'N/A')} total, {summary.get('valid_setup_keys', 'N/A')} valid")

print("\n✅ NetBird Python Client Demo Complete!")

📈 NetBird Network Summary
👥 Users:          4 total, 2 admins, 4 active
🖥️  Peers:          4 total, 2 online, 2 offline
👥 Groups:         4
🔐 Policies:       4 total, 4 enabled
🛣️  Routes:         0
🔑 Setup Keys:     3 total, 3 valid

✅ NetBird Python Client Demo Complete!


## 🎓 Next Steps

Now that you've seen the NetBird Python client in action, here are some ideas for your own projects:

### 🔧 Automation Scripts
- Automatically add new devices to specific groups
- Monitor peer connectivity and send alerts
- Backup and restore network configurations

### 📊 Monitoring & Analytics
- Generate network usage reports
- Track peer connection patterns
- Monitor policy effectiveness

### 🌐 Integration Projects
- Integrate with infrastructure automation tools
- Build custom dashboards
- Create API webhooks for real-time updates

### 📚 Resources
- **Documentation**: Check the `/tests/` folder for more examples
- **API Reference**: NetBird API documentation
- **GitHub**: Contribute to the project or report issues

---

**Happy networking with NetBird! 🚀**

In [28]:
users

[{'auto_groups': [],
  'email': 'bhushan.rane@cloudpoet.in',
  'id': '103489590992358909976',
  'is_blocked': False,
  'is_current': True,
  'is_service_user': False,
  'issued': 'api',
  'last_login': '2025-07-21T16:56:05.42356783Z',
  'name': 'Bhushan Rane',
  'role': 'owner',
  'status': 'active'},
 {'auto_groups': [],
  'email': '',
  'id': '6edb741a-5c62-40be-9143-7e514dbc7c7a',
  'is_blocked': False,
  'is_current': False,
  'is_service_user': True,
  'issued': '',
  'last_login': '0001-01-01T00:00:00Z',
  'name': 'terraform-admin',
  'role': 'admin',
  'status': 'active'},
 {'auto_groups': [],
  'email': 'shreya.fegade@cloudpoet.in',
  'id': '109338223303262796331',
  'is_blocked': False,
  'is_current': False,
  'is_service_user': False,
  'issued': 'api',
  'last_login': '2025-07-07T17:35:05.689222142Z',
  'name': 'Shreya Fegade',
  'role': 'user',
  'status': 'active'},
 {'auto_groups': [],
  'email': 'mahendra.bendale@cloudpoet.in',
  'id': '105863599789504595500',
  'is_blo

In [29]:
peers

[{'accessible_peers_count': 0,
  'approval_required': False,
  'city_name': 'Mumbai',
  'connected': True,
  'connection_ip': '144.24.116.110',
  'country_code': 'IN',
  'dns_label': 'rb-oci-k8s.netbird.selfhosted',
  'ephemeral': False,
  'extra_dns_labels': [],
  'geoname_id': 1275339,
  'groups': [{'id': 'd1ip995aofms73a23dp0',
    'name': 'All',
    'peers_count': 4,
    'resources_count': 1},
   {'id': 'd1mcn8daofms738ms6pg',
    'name': 'rb-oci',
    'peers_count': 1,
    'resources_count': 2}],
  'hostname': 'rb-oci-k8s',
  'id': 'd1mcpadaofms738ms6r0',
  'inactivity_expiration_enabled': False,
  'ip': '100.107.174.91',
  'kernel_version': '6.8.0-1025-oracle',
  'last_login': '2025-07-08T07:45:45.868423035Z',
  'last_seen': '2025-07-15T07:44:28.888472203Z',
  'login_expiration_enabled': False,
  'login_expired': False,
  'name': 'rb-oci-k8s',
  'os': 'Alpine Linux 3.21.3',
  'serial_number': '',
  'ssh_enabled': False,
  'ui_version': '',
  'user_id': '',
  'version': '0.50.1'},